# EPANET-Turbo Linux Build Verification (M5)

This notebook verifies the compilation and runtime of EPANET-Turbo on a Linux environment (Google Colab/Ubuntu).

**Objective**: Build `libepanet2.so` from **Private Source** (uploaded to Google Drive). Includes robust source path detection.

In [ ]:
# 1. Install Prerequisites
!apt-get update
!apt-get install -y cmake build-essential libomp-dev

## 2. Load Source from Google Drive

Assumes you have uploaded the `epanet-turbo` folder to your Google Drive.

In [ ]:
from google.colab import drive
import os
import shutil
import glob

# Mount Drive
drive.mount('/content/drive')

# --- CONFIGURATION ---
# Folder name in Google Drive
drive_folder_name = "epanet-turbo"
drive_source_path = f"/content/drive/MyDrive/{drive_folder_name}"
local_source_path = "/content/epanet-turbo"
# ---------------------

if os.path.exists(local_source_path):
    shutil.rmtree(local_source_path)

print(f"Copying {drive_source_path} -> {local_source_path} ...")
try:
    shutil.copytree(drive_source_path, local_source_path)
    print("✅ Core files copied.")
except FileNotFoundError:
    print(f"❌ Error: Could not find '{drive_source_path}'. Check your Drive folder name!")
    raise

%cd {local_source_path}

# --- INTELLIGENT SOURCE REPAIR ---
# Find where 'epanet2.c' is hiding
found_files = glob.glob(f"**/**/epanet2.c", recursive=True)

if not found_files:
    print("❌ CRITICAL ERROR: epanet2.c not found! Did you upload the full source (including private_src)?")
    print("Current directory structure:")
    !find .
else:
    target_src = "src"
    actual_src_file = found_files[0]
    actual_src_dir = os.path.dirname(actual_src_file)
    
    print(f"🔍 Found core source at: {actual_src_dir}")
    
    # If it is 'private_src' or nested, move it to 'src'
    if actual_src_dir != target_src:
        if os.path.exists(target_src):
             print(f"⚠️ Removing existing empty/wrong '{target_src}'...")
             shutil.rmtree(target_src)
        
        print(f"🔧 Renaming '{actual_src_dir}' to '{target_src}' for CMake compatibility...")
        shutil.move(actual_src_dir, target_src)
        print("✅ Source structure repaired.")
    else:
        print("✅ Source structure already correct.")

!ls -la src/ | head -n 5

## 3. Build with CMake

In [ ]:
!rm -rf build
!mkdir -p build
%cd build
!cmake ..
!make -j4

## 4. Verify Binaries

In [ ]:
!ls -lh *.so

## 5. Python Integration Test

In [ ]:
%cd /content/epanet-turbo
%pip install polars numpy

import ctypes
import os

# Manually load the built shared libraries for testing
so_path_serial = "./build/libepanet2.so"
so_path_openmp = "./build/libepanet2_openmp.so"

def verify_so(path, name):
    if not os.path.exists(path):
        print(f"❌ {name} not found at {path}")
        return
    
    try:
        lib = ctypes.CDLL(path)
        # Check ENT_engine_id
        if hasattr(lib, "ENT_engine_id"):
             lib.ENT_engine_id.restype = ctypes.c_char_p
             eid = lib.ENT_engine_id().decode()
             print(f"✅ {name}: ENT_engine_id = {eid}")
        else:
             print(f"⚠️ {name}: ENT_engine_id not found (Old version?)")
             
        # Check ENT_set_node_values (Batch API)
        if hasattr(lib, "ENT_set_node_values"):
            print(f"✅ {name}: Batch API found")
        else:
            print(f"❌ {name}: Batch API missing")
            
    except Exception as e:
        print(f"❌ {name} load failed: {e}")

verify_so(so_path_serial, "Serial Engine")
verify_so(so_path_openmp, "OpenMP Engine")